In [1]:
import sys
import os
NOTEBOOK_DIR = os.getcwd()
sys.path.append(os.path.abspath(os.path.join(NOTEBOOK_DIR, '..')))

In [2]:
# !pip install wget

In [3]:
# !pip install navec

In [4]:
# import wget
# url = 'https://storage.yandexcloud.net/natasha-navec/packs/navec_hudlit_v1_12B_500K_300d_100q.tar'
# wget.download(url)

In [5]:
import nltk
import pymorphy3
import scipy
import numpy as np

from navec import Navec

from utils import dataset_utils
from utils import metrics

# Search by word embeddings

## Load Data

In [6]:
REQUESTS_FILE = "../data/request_db.txt"
ADS_FILE = "../data/ads_db.txt"
MATCHING_FILE = "../data/matching_db.txt"

In [7]:
with open(ADS_FILE, encoding="utf-8") as f:
    ads_raw = f.readlines()

In [8]:
with open(REQUESTS_FILE, encoding="utf-8") as f:
    requests_raw = f.readlines()

In [9]:
true_markup = dataset_utils.load_matching_data(MATCHING_FILE)

## Preprocessing

In [10]:
def preprocess(text):
    text = text.replace("\\n", "\n").replace("\n", " ")
    text = text.strip()
    text = text.lower()  # many words have vectors only in lowercase
    return text


def tokenize(text):
    return nltk.tokenize.word_tokenize(text)


def remove_stop_words(tokens, stop_word_list):
    return [tok for tok in tokens if tok not in stop_word_list]


def make_normal_forms(morph, tokens):
    return [morph.parse(tok)[0].normal_form for tok in tokens]


def encode_tokens_to_vectors(navec_model, token_list):
    vec_list = []
    for tokens in token_list:
        vec_list.append([navec_model[tok] for tok in tokens if tok in navec_model])
    return vec_list

In [11]:
nltk.download("stopwords")

True

In [12]:
nltk.download("punkt_tab")

True

In [13]:
rus_stop_words = nltk.corpus.stopwords.words("russian")

In [14]:
morph = pymorphy3.MorphAnalyzer()

In [15]:
ad_tokens = [make_normal_forms(morph, remove_stop_words(tokenize(preprocess(text)), rus_stop_words)) for text in ads_raw]

In [16]:
req_tokens = [make_normal_forms(morph, remove_stop_words(tokenize(preprocess(text)), rus_stop_words)) for text in requests_raw]

In [17]:
navec_model = Navec.load("navec_hudlit_v1_12B_500K_300d_100q.tar")
# navec_model = Navec.load("navec_news_v1_1B_250K_300d_100q.tar")

In [18]:
enc_ad_tokens = encode_tokens_to_vectors(navec_model, ad_tokens)

In [19]:
enc_req_tokens = encode_tokens_to_vectors(navec_model, req_tokens)

## Prediction

In [20]:
def predict_by_vectors(enc_req_tok_list, enc_ad_tok_list, threshold):
    def cosine_dist(x, y):
        return scipy.spatial.distance.cosine(x, y)/2

    predictions = {}
    for req_id, enc_req_tokens in enumerate(enc_req_tok_list, start=1):
        found_list = []
        for ad_id, enc_ad_tokens in enumerate(enc_ad_tok_list, start=1):
            if len(enc_req_tokens) == 0:
                continue
            if all(any(cosine_dist(req_tok, ad_tok) <= threshold for ad_tok in enc_ad_tokens) for req_tok in enc_req_tokens):
                found_list.append(str(ad_id))
        if len(found_list) > 0:
            predictions[str(req_id)] = found_list.copy()
    return predictions

In [21]:
def get_probs(enc_req_tok_list, enc_ad_tok_list, dist_type):
    def cosine_sim(x, y):
        dst = scipy.spatial.distance.cosine(x, y)
        assert dst >= 0
        assert dst <= 2
        return 1 - (dst/2)

    def euc_sim(x, y):
        dst = np.sqrt(np.sum((x-y)**2))/(np.sqrt(np.sum(x**2)) + np.sqrt(np.sum(y**2)))
        assert dst >= 0
        assert dst <= 1
        return 1 - dst

    if dist_type == 'cosine':
        sim_func = cosine_sim
    elif dist_type == 'euclidean':
        sim_func = euc_sim
    else:
        raise ValueError(f"Unknown distance type: {dist_type}")

    all_probs = []
    for enc_req_tokens in enc_req_tok_list:
        probs = []
        for enc_ad_tokens in enc_ad_tok_list:
            probs.append(min([max([sim_func(req_tok, ad_tok) for ad_tok in enc_ad_tokens], default=0) for req_tok in enc_req_tokens], default=0))
        all_probs.append(probs.copy())
    return np.asarray(all_probs)

### Tries with Manual Threshold

In [22]:
# pred_markup = predict_by_vectors(enc_req_tokens, enc_ad_tokens, 0.6)
# pred_markup = predict_by_vectors(enc_req_tokens, enc_ad_tokens, 0.25)
# pred_markup = predict_by_vectors(enc_req_tokens, enc_ad_tokens, 0.1)
pred_markup = predict_by_vectors(enc_req_tokens, enc_ad_tokens, 1 - 0.8551743030548096)

In [23]:
confusion_matrix = metrics.calc_confusion_matrix(true_markup, pred_markup, n_ads=len(ads_raw), n_requests=len(requests_raw))
confusion_matrix

{'TP': 106, 'FP': 128, 'TN': 87100, 'FN': 488}

In [24]:
stats = metrics.calc_all_stats(confusion_matrix)
stats

{'accuracy': 0.9929858122110633,
 'precision': 0.452991452991453,
 'recall': 0.17845117845117844,
 'f1': 0.2560386473429952}

In [25]:
metrics.compare_with_saved_stats(stats, confusion_matrix)

-----------------------------------------------------------------------------------------
|	Metric		|	Old Value	|	New Value	|	Diff	|
-----------------------------------------------------------------------------------------
|	TP		|	216		|	106		|	📉 -110	|
|	FP		|	418		|	128		|	📉 -290	|
|	TN		|	86810		|	87100		|	📈 290	|
|	FN		|	378		|	488		|	📈 110	|
|	Prec		|	0.341		|	0.453		|	📈 0.112	|
|	Recall		|	0.364		|	0.178		|	📉 -0.185	|
|	F1		|	0.352		|	0.256		|	📉 -0.096	|

F1 📉 decreased by 0.096, down to 25.6%, which is a significant fall.


### Tries with Automatic Threshold

#### Cosine Distance

In [26]:
all_probs = get_probs(enc_req_tokens, enc_ad_tokens, 'cosine')
opt_threshold = metrics.calc_optimal_threshold(all_probs, true_markup, len(requests_raw), len(ads_raw))
print(f"Optimal threshold for cosine distance: {opt_threshold}")
pred_markup = metrics.convert_probs_to_markup(all_probs, opt_threshold, len(requests_raw), len(ads_raw))

Optimal threshold for cosine distance: 0.8551743030548096


In [27]:
confusion_matrix = metrics.calc_confusion_matrix(true_markup, pred_markup, n_ads=len(ads_raw), n_requests=len(requests_raw))
confusion_matrix

{'TP': 106, 'FP': 128, 'TN': 87100, 'FN': 488}

In [28]:
stats = metrics.calc_all_stats(confusion_matrix)
stats

{'accuracy': 0.9929858122110633,
 'precision': 0.452991452991453,
 'recall': 0.17845117845117844,
 'f1': 0.2560386473429952}

In [29]:
metrics.compare_with_saved_stats(stats, confusion_matrix)

-----------------------------------------------------------------------------------------
|	Metric		|	Old Value	|	New Value	|	Diff	|
-----------------------------------------------------------------------------------------
|	TP		|	216		|	106		|	📉 -110	|
|	FP		|	418		|	128		|	📉 -290	|
|	TN		|	86810		|	87100		|	📈 290	|
|	FN		|	378		|	488		|	📈 110	|
|	Prec		|	0.341		|	0.453		|	📈 0.112	|
|	Recall		|	0.364		|	0.178		|	📉 -0.185	|
|	F1		|	0.352		|	0.256		|	📉 -0.096	|

F1 📉 decreased by 0.096, down to 25.6%, which is a significant fall.


#### Euclidean Distance

In [30]:
all_probs = get_probs(enc_req_tokens, enc_ad_tokens, 'euclidean')
opt_threshold = metrics.calc_optimal_threshold(all_probs, true_markup, len(requests_raw), len(ads_raw))
print(f"Optimal threshold for euclidean distance: {opt_threshold}")
pred_markup = metrics.convert_probs_to_markup(all_probs, opt_threshold, len(requests_raw), len(ads_raw))

Optimal threshold for euclidean distance: 0.6187620162963867


In [31]:
confusion_matrix = metrics.calc_confusion_matrix(true_markup, pred_markup, n_ads=len(ads_raw), n_requests=len(requests_raw))
confusion_matrix

{'TP': 106, 'FP': 125, 'TN': 87103, 'FN': 488}

In [32]:
stats = metrics.calc_all_stats(confusion_matrix)
stats

{'accuracy': 0.9930199722165289,
 'precision': 0.4588744588744589,
 'recall': 0.17845117845117844,
 'f1': 0.25696969696969696}

In [33]:
metrics.compare_with_saved_stats(stats, confusion_matrix)

-----------------------------------------------------------------------------------------
|	Metric		|	Old Value	|	New Value	|	Diff	|
-----------------------------------------------------------------------------------------
|	TP		|	216		|	106		|	📉 -110	|
|	FP		|	418		|	125		|	📉 -293	|
|	TN		|	86810		|	87103		|	📈 293	|
|	FN		|	378		|	488		|	📈 110	|
|	Prec		|	0.341		|	0.459		|	📈 0.118	|
|	Recall		|	0.364		|	0.178		|	📉 -0.185	|
|	F1		|	0.352		|	0.257		|	📉 -0.095	|

F1 📉 decreased by 0.095, down to 25.7%, which is a significant fall.


## Requests with no vectors

Maybe, the reason of low accuracy is requests that have no word embeddings? There are words with intentionally added typos and just usual OOV words. Words with typos won't work even for direct matching, so we should consider only OOV words. Let's calculate decrease in `TP` due to OOV words.

In [34]:
not_found_words = []
for req_tok_list in req_tokens:
    for tok in req_tok_list:
        try:
            navec_model[tok]
        except KeyError:
            not_found_words.append(tok)
print("Words without embeddings:\n" + "\n".join(sorted(set(not_found_words))))

Words without embeddings:
-30
.
1
12
160см
3
30см
38
42
45к
48
5
50-52
5м
6
64
68-74
7
автокресло
автолюлька
акссесуар
беговеть
безсахар
большша
вейп
велотренажёр
весна-осень
деми-сезон
демисезон
демисезонный
дещево
дкарансть
драповый
жеский
зеркалка
зима-осень
икея
искуственный
кастюм
клининга
кожзам
комбинзон
кометик
короткий-приталенный
котоняня
крусло
крутка
мощьный
оверсайза
огэ
односпальный
онтоновка
опилка
пальто-куртка
по-английскому
приталёный
распашенка
расстущий
ротанг
сайза
самовывоз
самосбор
сандали
снуда
совесткий
соцгород
сумка-шопер
сушёный
телпый
туфилька
уходовой
фотостудия
чермета
шифонёр
шотр
экокожа
экомешочек
электросамокат


In [35]:
# numbers were also skipped, since we don't expect they appear in the advertisements
real_oov_words = [
    "автокресло", "автолюлька", "акссесуар", "беговеть", "безсахар", "большша", "вейп", "велотренажёр", "весна-осень", "деми-сезон",
    "демисезон", "демисезонный", "дещево", "дкарансть", "драповый", "жеский",
    "зеркалка", "зима-осень", "икея", "искуственный", "кастюм", "клининга", "кожзам", "комбинзон", "кометик", "короткий-приталенный", "котоняня", 
    "крусло", "крутка", "мощьный", "оверсайза", "огэ", "односпальный", "онтоновка",
    "опилка", "пальто-куртка", "по-английскому", "приталёный", "распашенка", "расстущий", "ротанг", 
    "сайза", "самовывоз", "самосбор", "сандали", "снуда","совесткий", "соцгород", "сушёный", "сумка-шопер", "телпый", "туфилька", "уходовой",
    "фотостудия", "чермета", "шифонёр", "шотр", "экокожа", "экомешочек", "электросамокат",
]

In [36]:
missed_tps = 0
for req_id, req_tok_list in enumerate(req_tokens, start=1):
    if any(tok in real_oov_words for tok in req_tok_list) and str(req_id) in true_markup:
        missed_tps += sum(
            1 if all(req_tok in ad_tokens[int(ad_id) - 1] for req_tok in req_tok_list) else 0 for ad_id in true_markup[str(req_id)]
        )
print(f"Missed TPs: {missed_tps}")

# comparing with cosine distance, for example
new_tp = 106 + missed_tps
new_fp = 128
new_tn = 87100 - missed_tps
new_fn = 488
print(f"New value for F1 would be {100*(2*(new_tp)/((2*new_tp) + new_fp + new_fn)):.1f}%, comparing with 25.6% for cosine distance")

Missed TPs: 7
New value for F1 would be 26.8%, comparing with 25.6% for cosine distance


So it looks like NaVec vectors don't help mainly because of lots of false positives, not only because of OOV words.